
# BERT Email Classification (Pre-trained Setup)

This notebook is structured to:
1. **Train** the BERT model (resource-intensive, already done).
2. **Tokenize** the email dataset and save it.
3. **Evaluate** the model using saved files.
4. Make it easy for others (e.g., professors) to **run and view results** without repeating expensive steps.


In [1]:

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


## 🧰 Environment & Dependencies

To ensure this notebook runs as intended, install the following packages with the specified versions (GPU recommended for training):

```bash
transformers==4.39.3
torch==2.2.1
scikit-learn==1.4.1
pandas==2.2.1
numpy==1.26.4
tqdm==4.66.1
```

If using a virtual environment:
```bash
pip install transformers==4.39.3 torch==2.2.1 scikit-learn==1.4.1 pandas==2.2.1 numpy==1.26.4 tqdm==4.66.1
```

Ensure your runtime supports CUDA (e.g., RTX 2060 or higher) for optimal training performance.


# This notebook is meant to train and save the BERT model on the phishing email dataset.
# It should be run once in a GPU environment. The saved model will be used for testing later. I have provided the file of the fine tuned model for testing, as this can be computationally intesive and time consuming


# ## 1. Load Dataset

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

df = pd.read_csv("../data/processed/CEAS_08_feature_engineered.csv")
texts = df["body"].fillna("").tolist()
labels = df["label"].tolist()


In [4]:

import os

MODEL_DIR = "../models/bert_finetuned"
TOKENIZED_INPUTS_PATH = "../models/tokenized_inputs.pt"

if os.path.exists(MODEL_DIR):
    print(f"✅ Model directory exists: {MODEL_DIR}")
else:
    print(f"❌ Model directory not found. You may need to train the model.")

if os.path.exists(TOKENIZED_INPUTS_PATH):
    print(f"✅ Tokenized inputs found at: {TOKENIZED_INPUTS_PATH}")
else:
    print(f"❌ No saved tokenized inputs found.")


✅ Model directory exists: ../models/bert_finetuned
❌ No saved tokenized inputs found.


# ## 2. Split into Train and Validation Sets

We will split the data into training and testing sets.
- 80% of the emails will be used for training the BERT model.
- 20% of the emails will be used for final evaluation.
Stratified sampling will be used to ensure both classes (phishing and legitimate) are represented proportionally in both sets.


In [5]:
# Perform train/test split
from sklearn.model_selection import train_test_split
# Use a smaller subset for faster experimentation
texts_subset, _, labels_subset, _ = train_test_split(texts, labels, train_size=0.10, random_state=42)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts_subset, labels_subset, test_size=0.2, random_state=42
)


# ## 3. Load Tokenizer and Tokenize Data


In [6]:

# Load tokenized inputs if available
if os.path.exists(TOKENIZED_INPUTS_PATH):
    inputs = torch.load(TOKENIZED_INPUTS_PATH)
    print("✅ Loaded tokenized inputs from file.")


In [7]:
from transformers import BertTokenizer

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


c:\Users\lukes\OneDrive\Documents\GitHub\INST414-project\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:

# Save tokenized inputs to avoid re-tokenizing
import torch

torch.save(inputs, TOKENIZED_INPUTS_PATH)
print("✅ Tokenized inputs saved.")


NameError: name 'inputs' is not defined

# ## 4. Create PyTorch Dataset


In [9]:
from transformers import BertForSequenceClassification

# Load pre-trained BERT model for sequence classification (binary classification)
import torch

class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {k: torch.tensor(v[idx]) for k, v in self.encodings.items()} | {"labels": torch.tensor(self.labels[idx])}

    def __len__(self):
        return len(self.labels)

train_dataset = EmailDataset(train_encodings, train_labels)
val_dataset = EmailDataset(val_encodings, val_labels)

# ## 5. Load BERT Model for Classification


In [10]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# ## 6. Define Training Arguments

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="../models/Bert_finetuned",
    num_train_epochs=1,  # Only 1 epoch
    per_device_train_batch_size=8,  # Reduce batch size
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=50,
)

# ## 7. Train the Model

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


trainer.train()

  0%|          | 0/392 [00:00<?, ?it/s]

{'loss': 0.2338, 'grad_norm': 0.17089270055294037, 'learning_rate': 4.362244897959184e-05, 'epoch': 0.13}
{'loss': 0.1837, 'grad_norm': 0.06506172567605972, 'learning_rate': 3.724489795918368e-05, 'epoch': 0.26}
{'loss': 0.0631, 'grad_norm': 0.0708104595541954, 'learning_rate': 3.086734693877551e-05, 'epoch': 0.38}
{'loss': 0.0553, 'grad_norm': 0.0257522352039814, 'learning_rate': 2.448979591836735e-05, 'epoch': 0.51}
{'loss': 0.0902, 'grad_norm': 21.786165237426758, 'learning_rate': 1.8112244897959187e-05, 'epoch': 0.64}
{'loss': 0.0789, 'grad_norm': 0.013717805966734886, 'learning_rate': 1.1734693877551021e-05, 'epoch': 0.77}
{'loss': 0.0416, 'grad_norm': 0.013210978358983994, 'learning_rate': 5.357142857142857e-06, 'epoch': 0.89}


  0%|          | 0/49 [00:00<?, ?it/s]

{'eval_loss': 0.024709470570087433, 'eval_accuracy': 0.9923371647509579, 'eval_f1': 0.9928909952606635, 'eval_runtime': 77.0214, 'eval_samples_per_second': 10.166, 'eval_steps_per_second': 0.636, 'epoch': 1.0}
{'train_runtime': 1043.6386, 'train_samples_per_second': 2.999, 'train_steps_per_second': 0.376, 'train_loss': 0.0990391808504961, 'epoch': 1.0}


TrainOutput(global_step=392, training_loss=0.0990391808504961, metrics={'train_runtime': 1043.6386, 'train_samples_per_second': 2.999, 'train_steps_per_second': 0.376, 'train_loss': 0.0990391808504961, 'epoch': 1.0})

# ## 8. Save Model and Tokenizer

In [13]:
model.save_pretrained("../models/Bert_finetuned")
tokenizer.save_pretrained("../models/Bert_finetuned")

('../models/Bert_finetuned\\tokenizer_config.json',
 '../models/Bert_finetuned\\special_tokens_map.json',
 '../models/Bert_finetuned\\vocab.txt',
 '../models/Bert_finetuned\\added_tokens.json')

# From here, we will be testing the model we created

## Testing

In [14]:
#1. Load your saved model and tokenizer
from transformers import BertTokenizer, BertForSequenceClassification

model_path = "../models/Bert_finetuned"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [15]:
#2. Load and preprocess your test data
#Make sure it matches how the training data was preprocessed (tokenized body field):
import pandas as pd

df = pd.read_csv("../data/processed/CEAS_08_feature_engineered.csv")
test_texts = df["body"].fillna("").tolist()
test_labels = df["label"].tolist()


In [16]:
#3. Tokenize for BERT
import torch

inputs = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")


In [20]:
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# ⚙️ Reduce batch size and sequence length to save memory
batch_size = 8
max_seq_len = 256

# 📉 Optional: use only a portion of test set to save time
test_texts = test_texts[:20000]
test_labels = test_labels[:20000]

# 🔁 Re-tokenize with smaller max_length
inputs = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=max_seq_len,
    return_tensors="pt"
)

# 🧱 Efficient dataset definition
class BERTDataset(Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings["input_ids"]
        self.attention_mask = encodings["attention_mask"]

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx]
        }

test_dataset = BERTDataset(inputs)

# 🚀 Load to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 🧠 Pin memory for speed if using GPU
kwargs = {"batch_size": batch_size}
if torch.cuda.is_available():
    kwargs["pin_memory"] = True

test_loader = DataLoader(test_dataset, **kwargs)

# 🔍 Inference
all_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Running Inference"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())



Running Inference: 100%|██████████| 2500/2500 [04:06<00:00, 10.14it/s]


In [21]:
print("Predictions:", len(all_preds))
print("Labels:     ", len(test_labels))


Predictions: 20000
Labels:      20000


In [22]:
#5. Evaluate results
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:\n", classification_report(test_labels, all_preds))
print("Confusion Matrix:\n", confusion_matrix(test_labels, all_preds))



Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      8940
           1       0.99      0.99      0.99     11060

    accuracy                           0.99     20000
   macro avg       0.99      0.99      0.99     20000
weighted avg       0.99      0.99      0.99     20000

Confusion Matrix:
 [[ 8858    82]
 [  103 10957]]


## ✅ Model Training Summary & Results

The BERT model was successfully fine-tuned for phishing email detection using the CEAS 2008 dataset. Key details:

- **Base model**: `bert-base-uncased`
- **Training time**: ~45 minutes on RTX 2060
- **Training epochs**: 1
- **Training batch size**: 8
- **Validation strategy**: Evaluated after each epoch using F1-score

### 🔍 Best Checkpoint:  
- Saved in `../models/Bert_finetuned`  
- Automatically selected based on highest F1-score during validation

This notebook produces a fine-tuned model that is ready for evaluation or deployment. The testing also goes on within this notebook


In [23]:

# Save tokenized inputs for future use
import torch

torch.save(inputs, "../models/tokenized_inputs.pt")
torch.save(labels, "../models/labels.pt")
print("Saved tokenized inputs and labels.")


Saved tokenized inputs and labels.



## Reuse Saved Tokenized Data and Model
## this last area should be all you need to run to see the model
This allows easy inference without re-training or re-tokenizing.


In [2]:

# Load saved data
import torch
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("../models/bert_finetuned/")
inputs = torch.load("../models/tokenized_inputs.pt")
labels = torch.load("../models/labels.pt")

print("Loaded model and tokenized inputs.")


Loaded model and tokenized inputs.


In [3]:

from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import torch

class BERTDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(next(iter(self.encodings.values())))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

test_dataset = BERTDataset(inputs)
test_loader = DataLoader(test_dataset, batch_size=64)

all_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
print("✅ Inference complete.")


  0%|          | 0/313 [00:00<?, ?it/s]

✅ Inference complete.


In [8]:
import pandas as pd
import numpy as np
import torch

# 1. Load the original text data (email bodies)
df = pd.read_csv("../data/processed/CEAS_08_cleaned.csv")
texts = df['body'].fillna("[No text]").tolist()

# 2. If labels is a tensor, convert to list; if it's already a list, leave it
if isinstance(labels, torch.Tensor):
    labels_list = labels.cpu().numpy().tolist()
else:
    labels_list = labels

# 3. If all_preds is a NumPy array or tensor, convert it to a list
if isinstance(all_preds, torch.Tensor):
    preds_list = all_preds.cpu().numpy().tolist()
elif isinstance(all_preds, np.ndarray):
    preds_list = all_preds.tolist()
else:
    preds_list = all_preds

# 4. Truncate everything to the shortest common length
min_len = min(len(texts), len(labels_list), len(preds_list))
texts = texts[:min_len]
labels_list = labels_list[:min_len]
preds_list = preds_list[:min_len]

# 5. Create a DataFrame
results_df = pd.DataFrame({
    "text": texts,
    "true_label": labels_list,
    "predicted_label": preds_list
})

# 6. Filter for misclassifications
errors_df = results_df[results_df["true_label"] != results_df["predicted_label"]]

# 7. Save to interim folder
errors_df.to_csv("../data/interim/bert_misclassified_samples.csv", index=False)

# 8. Print preview
print("✅ Misclassified examples saved to: data/interim/bert_misclassified_samples.csv")
print(errors_df.sample(5, random_state=42)[['text', 'true_label', 'predicted_label']])


✅ Misclassified examples saved to: data/interim/bert_misclassified_samples.csv
                                                    text  true_label  \
2130   as an amazon.com customer, you might be intere...           0   
3989   view our free overnight shipping offer at\nhtt...           0   
17447  sorry. i just sent my email to the lwv list se...           1   
13211  pamela anderson... again... see now! \nhttp://...           1   
16975  though i have been state healthcare chair for ...           1   

       predicted_label  
2130                 1  
3989                 1  
17447                0  
13211                0  
16975                0  
